In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from IPython.core.display import display, HTML
from functools import reduce
from sklearn import metrics
#from nltk.corpus import stopwords

display(HTML("<style>.container { width:75% !important; }</style>"))
from spellchecker import SpellChecker 

import re
import splitter
rgx = re.compile("([\w][\w']*\w)")
import random
import splitter
import statistics
text = open("train.txt").readlines()
textLen = len(text)

In [2]:
#Decoupage du texte en cinq (k=5) sous parties
def textToPartions(text,k):
    portions=[]
    increments = round(textLen/k)
    for i in range(0,textLen,increments):
        portions.append((i,i+increments-1))
        
    return portions

In [3]:
def portionsToSubtext(textArray, portions):
    resText=[]
    for i,j in portions:
        for l in textArray[i:j+1]:
            resText.append(l)
            
    return resText

In [4]:
#Shuffle Text partitions
#Input number of (train, test, validation partitions) = 5 
def createShuffledSubText(originalText):
     
    partitions = textToPartions(originalText,5)
    result = { 'Train-Test': [], 'validation': partitions.pop()}
    
    for i in range(len(partitions)):
        
        curr = {'train' : [partitions[i],partitions[(i+1)%4],partitions[(i+2)%4]], 
                'test'  : [partitions[(i+3)%4]],
                'language':{}
                ,'words'  :{}
                , 'score':0, 'failed':[]}
        
        result['Train-Test'].append(curr)
    return result

In [5]:
def TF_IDF(corpus):
    
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus)
    #print(vectorizer.get_feature_names())
    #print(X.shape)
    return X.toarray()

In [6]:
#Test
"""
portions = textToPortions(text,100)
processedText = portionsToSubtext(text,portions)
print("Test passed" if len(processedText) == len(text) else "Failed")
"""

'\nportions = textToPortions(text,100)\nprocessedText = portionsToSubtext(text,portions)\nprint("Test passed" if len(processedText) == len(text) else "Failed")\n'

In [7]:
Data = createShuffledSubText(text)
Train_Test = Data['Train-Test']
Valid = Data['validation']
#Train_Test

In [8]:
def tokenize(input_text):
    
    # list of words
    # dict of words
    # total of phrases
    
    tokens_dict = {}
    tokens_list = []
    phrases_cnt = 0;
    
    for s in input_text.lower().split(". "):
        phrases_cnt +=1
        
        for w in rgx.findall(s):
            tokens_list.append(w)
            tokens_dict[w] = ((tokens_dict[w][0])+1,'Pure') if w in tokens_dict else (1,'Pure')
            
    return tokens_list, tokens_dict, phrases_cnt

In [9]:
def wordCounter(row,language,TTP):
    
    # Input the corpus as an array of inputs paragraphes, a target language, returns:
    # list of (length of words in a dictionary, [distinct words count, sentences count], the dictionary of words)
    
    tk = tokenize(row)
    for w in tk[1]:
        
        # Processing the lang dictionary
        #dict[w]= ((dict[w][0])+1,'Pure') if w in dict else (1,'Pure')
        
        w_occurence = tk[1][w][0]  
        #print(w_occurence)
        #Processing the global dictionary
            
        if w not in TTP['words']: 
                               
            TTP['words'][w]= {'occurences':w_occurence,'language':{language: w_occurence }}
            
        elif w in TTP['words']:
            
            TTP['words'][w]['occurences']+= w_occurence
            
            if language not in TTP['words'][w]['language']:
                
                TTP['words'][w]['language'][language] = w_occurence
            else:
                #print(TTP['words'][w][language])
                TTP['words'][w]['language'][language]+= w_occurence
            
    return tk[0], tk[1], tk[2], TTP

In [10]:
def addDict(d1, d2):
    for w in d2:
        if w not in d1:
            d1[w]=d2[w]
        else: 
            d1[w]=((d1[w][0]+d2[w][0]),d1[w][1])
    return d1

In [11]:
#Processing all text inputs in all partitions
for TTP in Train_Test:
    
    TTP_Train = portionsToSubtext(text,TTP['train'])
    TTP['tf_idf'] = TF_IDF(TTP_Train)
    
    for i in range(len(TTP_Train)):
        
        #Processing language dictionnary of TTP partition
        lang = TTP_Train[i][1:4]
        wc = wordCounter(TTP_Train[i][5:-1],lang,TTP)
        TTP=wc[3]#???
       
        if lang in TTP['language']:

            TTP['language'][lang]['Dict'     ] = addDict(TTP['language'][lang]['Dict'],wc[1])
            TTP['language'][lang]['Words'    ]+= len(wc[0])      #List of all words in row
            TTP['language'][lang]['Phrases'  ]+= wc[2]           #Total of all sentences in row
            
        else :

            TTP['language'][lang] = {'Dict':wc[1],'Words':len(wc[0]),'Phrases':wc[2]}

In [12]:
#Processing all text inputs in all partitions
for TTP in Train_Test:
    for l in TTP['language']:
        w_count    = TTP['language'][l]['Words']
        p_count    = TTP['language'][l]['Phrases']
        d_distinct = len(TTP['language'][l]['Dict'])
        TTP['language'][l]['Distinct']= d_distinct
        
        TTP['language'][l]['avg_WP']= round(w_count/p_count,2)
        TTP['language'][l]['avg_DW']= round(d_distinct/w_count,2)
        TTP['language'][l]['avg_DP']= round(d_distinct/p_count,2)

In [13]:
#Enriching TTPs with mean and orther stats
for TT in Train_Test:
    #Taille Moyenne des phrase en mots
    print("Partition", TT['train'],'\n')
    for l in TT['language']:
        
        print(l,
              "Total Words:",   TT['language'][l]['Words'],
              "Distinct Words:",TT['language'][l]['Distinct'],
              "Total Phrases:", TT['language'][l]['Phrases'],
              "AVG-WP:",TT['language'][l]['avg_WP'],
              "AVG WD:",TT['language'][l]['avg_DP'],
              "Ratio DW:",TTP['language'][l]['avg_DW'])
    print("\n")

Partition [(0, 1979), (1980, 3959), (3960, 5939)] 

GER Total Words: 168567 Distinct Words: 8258 Total Phrases: 8732 AVG-WP: 19.3 AVG WD: 0.95 Ratio DW: 0.05
TUR Total Words: 166726 Distinct Words: 8506 Total Phrases: 9488 AVG-WP: 17.57 AVG WD: 0.9 Ratio DW: 0.05
CHI Total Words: 166209 Distinct Words: 9072 Total Phrases: 9396 AVG-WP: 17.69 AVG WD: 0.97 Ratio DW: 0.05
TEL Total Words: 175638 Distinct Words: 9943 Total Phrases: 8957 AVG-WP: 19.61 AVG WD: 1.11 Ratio DW: 0.06
ARA Total Words: 151002 Distinct Words: 10253 Total Phrases: 6685 AVG-WP: 22.59 AVG WD: 1.53 Ratio DW: 0.07
SPA Total Words: 167930 Distinct Words: 9224 Total Phrases: 7208 AVG-WP: 23.3 AVG WD: 1.28 Ratio DW: 0.05
HIN Total Words: 183841 Distinct Words: 10634 Total Phrases: 9288 AVG-WP: 19.79 AVG WD: 1.14 Ratio DW: 0.06
JPN Total Words: 150055 Distinct Words: 8017 Total Phrases: 9646 AVG-WP: 15.56 AVG WD: 0.83 Ratio DW: 0.05
KOR Total Words: 161829 Distinct Words: 8680 Total Phrases: 10145 AVG-WP: 15.95 AVG WD: 0.86 

In [14]:
def extract(a,b):
    
    for w in a:
        """
        if type(a[w])=='List' and a[w][1]=='Dirty':
            continue;
        if type(a[w])=='int' and w in b:
           
            a[w]=[a[w],'Dirty']         
        else :
            a[w]=[a[w],'Pure']
        """
        if a[w][1]=='Dirty':
            continue;
        elif w in b: 
            a[w]=(a[w][0],'Dirty')
    return a

In [15]:
for TT in Train_Test:
    
    langs = list(TT['language'].keys())
    size  = len(langs)

    for x in range(size):
        for y in range(size):
            if x != y:
                TT['language'][langs[x]]['D'] = extract(TT['language'][langs[x]]['Dict'],TT['language'][langs[y]]['Dict'])

In [16]:
#Train_Test[0]['words']['you']
#Train_Test[0]['language']['GER']['Dict']['you']

In [17]:
for TT in Train_Test:
    #Testing acuracy of both dictionaries 
    
    for word in TT['words'].keys() :
        for lang in TT['words'][word]['language'].keys():
            if word not in TT['language'][lang]['Dict']:
                print(TT['train'],word)
                break

In [18]:
def inter(l1, l2):
    dict = {}
    for e in l2:
        if e in l1:
            dict[e]=0
    return dict

In [80]:
def stats(TT,tokens,debug):
         
    langStats = {
        
        'FRE':{ 'pure':0, 'global':0, 'dirty':0 },
        'ARA':{ 'pure':0, 'global':0, 'dirty':0 },
        'CHI':{ 'pure':0, 'global':0, 'dirty':0 },
        'TUR':{ 'pure':0, 'global':0, 'dirty':0 },
        'TEL':{ 'pure':0, 'global':0, 'dirty':0 },
        'GER':{ 'pure':0, 'global':0, 'dirty':0 },
        'JPN':{ 'pure':0, 'global':0, 'dirty':0 },
        'HIN':{ 'pure':0, 'global':0, 'dirty':0 },
        'SPA':{ 'pure':0, 'global':0, 'dirty':0 },
        'KOR':{ 'pure':0, 'global':0, 'dirty':0 },
        'ITA':{ 'pure':0, 'global':0, 'dirty':0 }
    }
     
    for each in langStats :
        
        D = TT['language'][each]['Dict']
        for w in tokens:
            
            if w in D:
                
                normalizer = len(D)*len(tokens)
                
                if D[w][1]=='Pure':
                    langStats[each]['pure']+=round(1/normalizer,10)
                    
                elif D[w][1]=='Dirty':
                    
                    current = D[w][0]
                    others  = TT['words'][w]['occurences']-current
            
                    occ = (current/(others/10))/normalizer
                    langStats[each]['dirty']= round(occ,10) +round(langStats[each]['dirty'],10)
                        
            elif w in TT['words']:
                
                langStats[each]['global'] =round(1/(len(TT['words'])*len(tokens)), 10) +round(langStats[each]['global'],10)
                
        if debug :
            print(each,langStats[each])
        
    return langStats, hasSetPure, hasSetDirty, hasSetGlobal

In [130]:
def box(t,TTP,debug):
     
    langStats,hasSetPure,hasSetDirty,hasSetGlobal = stats(TTP,tokenize(t)[0],debug)
        
    return min(langStats.keys(), key=(lambda k: (1/(langStats[k]['pure']+1))*(langStats[k]['global']+1)/(langStats[k]['dirty']+1)))

In [131]:
def score (TT): 
    
    Test = portionsToSubtext(text,TT['test'])
    acc = 0
    failed = []
    L = len(Test)
    for i in range(L):

        T=Test[i]
        trueLang = T[1:4]

        predictedLang = box(T[5:-1],TT,False)
        if predictedLang != trueLang:
            #print(i,trueLang,predictedLang)
            failed.append(i)

        if predictedLang == trueLang:
            acc+=1/L
    return acc,failed

In [132]:
#Test with Cross Validation
for TT in Train_Test:
    TT['score'],TT['failed']=score(TT)

In [133]:
i = 1
for each in Train_Test:
    print("Portion",i, 'score',round(each['score']*100,2),'%')
    i+=1

Portion 1 score 51.41 %
Portion 2 score 50.35 %
Portion 3 score 50.76 %
Portion 4 score 48.94 %


In [134]:
#Analysing failed in first partition
for i in Train_Test[0]['failed']:
    Text = portionsToSubtext(text,Train_Test[0]['test'])
    
    print('---------------------------',Text[i][1:4],'<x>',box(Text[i],Train_Test[0],True),'\n')


FRE {'pure': 0, 'global': 9.549e-07, 'dirty': 0.0001073473}
ARA {'pure': 0, 'global': 4.244e-07, 'dirty': 8.957210000000001e-05}
CHI {'pure': 4.518e-07, 'global': 5.305e-07, 'dirty': 0.0001399694}
TUR {'pure': 0, 'global': 8.488e-07, 'dirty': 0.0001164592}
TEL {'pure': 0, 'global': 8.488e-07, 'dirty': 9.96029e-05}
GER {'pure': 0, 'global': 6.366000000000001e-07, 'dirty': 0.0001218486}
JPN {'pure': 0, 'global': 6.366000000000001e-07, 'dirty': 0.0001376062}
HIN {'pure': 0, 'global': 5.305e-07, 'dirty': 9.96733e-05}
SPA {'pure': 0, 'global': 8.488e-07, 'dirty': 9.915709999999999e-05}
KOR {'pure': 0, 'global': 7.427e-07, 'dirty': 0.00012473149999999998}
ITA {'pure': 0, 'global': 5.305e-07, 'dirty': 0.0001109184}
--------------------------- CHI <x> CHI 

FRE {'pure': 0, 'global': 7.350000000000001e-07, 'dirty': 0.00011895909999999999}
ARA {'pure': 0, 'global': 8.82e-07, 'dirty': 8.72246e-05}
CHI {'pure': 0, 'global': 4.41e-07, 'dirty': 0.0001125722}
TUR {'pure': 0, 'global': 5.88e-07, 'dirt

GER {'pure': 0, 'global': 5.543999999999999e-07, 'dirty': 0.0001220649}
JPN {'pure': 0, 'global': 3.6960000000000003e-07, 'dirty': 0.00012179959999999999}
HIN {'pure': 0, 'global': 4.62e-07, 'dirty': 0.0001052427}
SPA {'pure': 0, 'global': 9.24e-07, 'dirty': 9.72748e-05}
KOR {'pure': 0, 'global': 2.7719999999999997e-07, 'dirty': 0.00012021679999999999}
ITA {'pure': 0, 'global': 5.543999999999999e-07, 'dirty': 0.0001025276}
--------------------------- ARA <x> GER 

FRE {'pure': 0, 'global': 1.3245e-06, 'dirty': 0.0001254187}
ARA {'pure': 0, 'global': 1.0596e-06, 'dirty': 8.25018e-05}
CHI {'pure': 0, 'global': 1.2362000000000001e-06, 'dirty': 0.00011455860000000001}
TUR {'pure': 0, 'global': 1.1479e-06, 'dirty': 0.0001210349}
TEL {'pure': 0, 'global': 1.3245e-06, 'dirty': 0.00011814160000000001}
GER {'pure': 0, 'global': 1.6777e-06, 'dirty': 0.0001076864}
JPN {'pure': 0, 'global': 1.6777e-06, 'dirty': 0.0001043169}
HIN {'pure': 6.418e-07, 'global': 1.0596e-06, 'dirty': 0.0001162846}
SPA 

CHI {'pure': 0, 'global': 4.07e-07, 'dirty': 0.0001165191}
TUR {'pure': 0, 'global': 4.07e-07, 'dirty': 0.00011088189999999999}
TEL {'pure': 0, 'global': 3.256e-07, 'dirty': 0.0001020242}
GER {'pure': 0, 'global': 8.14e-08, 'dirty': 0.0001294998}
JPN {'pure': 0, 'global': 2.4420000000000003e-07, 'dirty': 0.0001233045}
HIN {'pure': 0, 'global': 4.07e-07, 'dirty': 0.0001015827}
SPA {'pure': 0, 'global': 2.4420000000000003e-07, 'dirty': 0.0001074274}
KOR {'pure': 0, 'global': 4.07e-07, 'dirty': 0.00011922340000000001}
ITA {'pure': 0, 'global': 2.4420000000000003e-07, 'dirty': 0.00011424949999999999}
--------------------------- FRE <x> GER 

FRE {'pure': 0, 'global': 1.0787e-06, 'dirty': 0.0001049712}
ARA {'pure': 0, 'global': 7.705000000000001e-07, 'dirty': 8.42813e-05}
CHI {'pure': 6.561e-07, 'global': 1.541e-07, 'dirty': 0.0001281786}
TUR {'pure': 6.998e-07, 'global': 6.164e-07, 'dirty': 0.0001122408}
TEL {'pure': 0, 'global': 7.705000000000001e-07, 'dirty': 9.80408e-05}
GER {'pure': 0,

JPN {'pure': 3.274e-07, 'global': 7.469e-07, 'dirty': 0.0001092373}
HIN {'pure': 0, 'global': 5.431999999999999e-07, 'dirty': 0.0001047076}
SPA {'pure': 0, 'global': 8.148e-07, 'dirty': 0.00011315210000000001}
KOR {'pure': 0, 'global': 9.506000000000001e-07, 'dirty': 0.0001115661}
ITA {'pure': 0, 'global': 4.7530000000000003e-07, 'dirty': 0.00011832820000000001}
--------------------------- TUR <x> TUR 

FRE {'pure': 0, 'global': 1.5199999999999998e-06, 'dirty': 0.000108477}
ARA {'pure': 4.578e-07, 'global': 1.0336e-06, 'dirty': 9.61989e-05}
CHI {'pure': 0, 'global': 1.2768e-06, 'dirty': 0.0001465846}
TUR {'pure': 2.76e-07, 'global': 1.6415999999999999e-06, 'dirty': 0.00011392459999999999}
TEL {'pure': 0, 'global': 1.6415999999999999e-06, 'dirty': 0.00011512}
GER {'pure': 0, 'global': 1.5808e-06, 'dirty': 0.0001182559}
JPN {'pure': 2.928e-07, 'global': 1.2768e-06, 'dirty': 0.00012561929999999998}
HIN {'pure': 0, 'global': 1.3376e-06, 'dirty': 0.0001053496}
SPA {'pure': 0, 'global': 1.51

--------------------------- KOR <x> GER 

FRE {'pure': 3.744e-07, 'global': 7.74e-07, 'dirty': 0.0001123113}
ARA {'pure': 0, 'global': 3.44e-07, 'dirty': 9.06735e-05}
CHI {'pure': 0, 'global': 6.02e-07, 'dirty': 0.0001076356}
TUR {'pure': 0, 'global': 6.88e-07, 'dirty': 0.00011914320000000001}
TEL {'pure': 0, 'global': 4.3e-07, 'dirty': 0.0001118529}
GER {'pure': 0, 'global': 6.88e-07, 'dirty': 0.000127067}
JPN {'pure': 4.144e-07, 'global': 6.88e-07, 'dirty': 0.00010207409999999999}
HIN {'pure': 0, 'global': 6.02e-07, 'dirty': 0.0001058357}
SPA {'pure': 0, 'global': 5.16e-07, 'dirty': 0.0001144133}
KOR {'pure': 0, 'global': 6.02e-07, 'dirty': 0.00011008280000000001}
ITA {'pure': 0, 'global': 6.02e-07, 'dirty': 0.00011673860000000001}
--------------------------- SPA <x> GER 

FRE {'pure': 0, 'global': 5.178e-07, 'dirty': 0.00015850990000000002}
ARA {'pure': 0, 'global': 7.767e-07, 'dirty': 8.01532e-05}
CHI {'pure': 0, 'global': 5.178e-07, 'dirty': 0.0001092583}
TUR {'pure': 3.919e-07, '

...

JPN {'pure': 0, 'global': 1.638e-06, 'dirty': 9.79586e-05}
HIN {'pure': 1.984e-07, 'global': 4.368e-07, 'dirty': 0.0001310158}
SPA {'pure': 2.287e-07, 'global': 1.0374e-06, 'dirty': 0.000102267}
KOR {'pure': 0, 'global': 9.828e-07, 'dirty': 0.0001049635}
ITA {'pure': 0, 'global': 1.2012e-06, 'dirty': 0.0001086469}
--------------------------- TEL <x> HIN 

FRE {'pure': 0, 'global': 3.144e-07, 'dirty': 0.0001116852}
ARA {'pure': 0, 'global': 3.144e-07, 'dirty': 8.58979e-05}
CHI {'pure': 4.463e-07, 'global': 3.144e-07, 'dirty': 0.0001133958}
TUR {'pure': 0, 'global': 2.096e-07, 'dirty': 0.00011190299999999999}
TEL {'pure': 0, 'global': 4.192e-07, 'dirty': 0.0001019091}
GER {'pure': 0, 'global': 3.144e-07, 'dirty': 0.00012316199999999998}
JPN {'pure': 0, 'global': 2.096e-07, 'dirty': 0.0001219804}
HIN {'pure': 0, 'global': 5.24e-07, 'dirty': 9.3449e-05}
SPA {'pure': 0, 'global': 3.144e-07, 'dirty': 0.00010411019999999999}
KOR {'pure': 0, 'global': 4.192e-07, 'dirty': 0.0001196615}
ITA {'pu

KOR {'pure': 0, 'global': 3.9269999999999996e-07, 'dirty': 0.00013075229999999998}
ITA {'pure': 0, 'global': 6.732e-07, 'dirty': 0.0001220064}
--------------------------- CHI <x> KOR 

FRE {'pure': 0, 'global': 6.240000000000001e-07, 'dirty': 0.00011808350000000001}
ARA {'pure': 0, 'global': 7.8e-07, 'dirty': 8.547900000000001e-05}
CHI {'pure': 0, 'global': 7.02e-07, 'dirty': 0.0001076205}
TUR {'pure': 0, 'global': 6.240000000000001e-07, 'dirty': 0.0001252129}
TEL {'pure': 0, 'global': 6.240000000000001e-07, 'dirty': 0.0001033594}
GER {'pure': 0, 'global': 7.8e-07, 'dirty': 0.0001291796}
JPN {'pure': 0, 'global': 8.580000000000001e-07, 'dirty': 0.00010810689999999999}
HIN {'pure': 0, 'global': 1.56e-07, 'dirty': 0.0001210046}
SPA {'pure': 0, 'global': 7.8e-07, 'dirty': 0.000111446}
KOR {'pure': 0, 'global': 4.68e-07, 'dirty': 0.0001127882}
ITA {'pure': 3.659e-07, 'global': 6.240000000000001e-07, 'dirty': 0.0001181969}
--------------------------- HIN <x> GER 

FRE {'pure': 0, 'global': 

CHI {'pure': 4.782e-07, 'global': 1.683e-07, 'dirty': 0.0001234264}
TUR {'pure': 0, 'global': 3.366e-07, 'dirty': 0.0001192943}
TEL {'pure': 0, 'global': 4.488e-07, 'dirty': 0.0001039906}
GER {'pure': 0, 'global': 3.366e-07, 'dirty': 0.0001227433}
JPN {'pure': 0, 'global': 3.366e-07, 'dirty': 0.0001118041}
HIN {'pure': 0, 'global': 4.488e-07, 'dirty': 9.89744e-05}
SPA {'pure': 0, 'global': 3.366e-07, 'dirty': 0.0001113496}
KOR {'pure': 0, 'global': 2.244e-07, 'dirty': 0.0001136418}
ITA {'pure': 2.635e-07, 'global': 4.488e-07, 'dirty': 0.0001094532}
--------------------------- CHI <x> CHI 

FRE {'pure': 0, 'global': 5.280000000000001e-07, 'dirty': 0.0001158662}
ARA {'pure': 0, 'global': 3.52e-07, 'dirty': 8.58376e-05}
CHI {'pure': 3.749e-07, 'global': 6.159999999999999e-07, 'dirty': 0.0001143742}
TUR {'pure': 0, 'global': 6.159999999999999e-07, 'dirty': 0.0001245156}
TEL {'pure': 3.421e-07, 'global': 7.04e-07, 'dirty': 9.45508e-05}
GER {'pure': 0, 'global': 6.159999999999999e-07, 'dirty

SPA {'pure': 0, 'global': 1.2628e-06, 'dirty': 0.0001148072}
KOR {'pure': 4.014e-07, 'global': 9.922e-07, 'dirty': 0.00012904629999999998}
ITA {'pure': 0, 'global': 1.6236e-06, 'dirty': 0.0001153001}
--------------------------- CHI <x> TUR 

FRE {'pure': 0, 'global': 4.16e-07, 'dirty': 0.0001189547}
ARA {'pure': 0, 'global': 7.488e-07, 'dirty': 8.40138e-05}
CHI {'pure': 3.544e-07, 'global': 8.32e-07, 'dirty': 0.000107569}
TUR {'pure': 0, 'global': 9.152e-07, 'dirty': 0.0001163468}
TEL {'pure': 0, 'global': 8.32e-07, 'dirty': 9.73831e-05}
GER {'pure': 0, 'global': 4.992e-07, 'dirty': 0.0001319716}
JPN {'pure': 0, 'global': 8.32e-07, 'dirty': 0.0001083007}
HIN {'pure': 0, 'global': 5.824e-07, 'dirty': 0.0001025907}
SPA {'pure': 0, 'global': 4.992e-07, 'dirty': 0.00010870859999999999}
KOR {'pure': 0, 'global': 6.656e-07, 'dirty': 0.00011032900000000001}
ITA {'pure': 0, 'global': 4.16e-07, 'dirty': 0.0001199835}
--------------------------- ITA <x> GER 

FRE {'pure': 0, 'global': 1.027e-06,

FRE {'pure': 7.246e-07, 'global': 1.2480000000000002e-06, 'dirty': 0.0001183678}
ARA {'pure': 0, 'global': 1.4975999999999999e-06, 'dirty': 9.48721e-05}
CHI {'pure': 7.088e-07, 'global': 1.3312e-06, 'dirty': 0.0001148986}
TUR {'pure': 0, 'global': 1.7472e-06, 'dirty': 0.0001129639}
TEL {'pure': 0, 'global': 1.7472e-06, 'dirty': 9.240290000000001e-05}
GER {'pure': 0, 'global': 1.4144e-06, 'dirty': 0.0001359}
JPN {'pure': 0, 'global': 1.9968e-06, 'dirty': 0.0001135435}
HIN {'pure': 3.024e-07, 'global': 1.2480000000000002e-06, 'dirty': 0.0001063883}
SPA {'pure': 0, 'global': 1.3312e-06, 'dirty': 0.0001194563}
KOR {'pure': 0, 'global': 1.8303999999999999e-06, 'dirty': 0.0001078168}
ITA {'pure': 0, 'global': 1.4975999999999999e-06, 'dirty': 0.0001133238}
--------------------------- FRE <x> GER 

FRE {'pure': 0, 'global': 1.0845e-06, 'dirty': 0.0001103394}
ARA {'pure': 0, 'global': 1.0122e-06, 'dirty': 9.46289e-05}
CHI {'pure': 0, 'global': 1.1567999999999999e-06, 'dirty': 0.0001165814}
TUR 

In [211]:
def createFeature(i,T,TTP):
    
    
    tk = tokenize(T[5:-1])
    Stats = stats(TTP,tk[0],False)[0]
    
    w   = len(tk[0])
    d   = len(tk[1])
    p   =     tk[2]
    
    dp  = round(d/p,2)
    wp  = round(w/p,2)
    dw  = round(d/w,2)
    wd  = round(w/d,2)
    
    features =[w,d,p,dp,wp,dw,wd]
      
    for s in Stats:
        
        P = Stats[s]['pure']  +0.1
        D = Stats[s]['dirty'] +0.1
        G = Stats[s]['global']+0.1
        
        features.append(P)
        features.append(G)     
        features.append(G/D)
        features.append(D)
        features.append((1/P)*(G/D))
        
        #features.append((P  )*(D/G))
        #features.append((1/P)*(D/G))
        
    #print(features,"\n")
    return features

In [ ]:
#Random Forest Classification with Cross Validation
for TT in Train_Test:
    
    Train = portionsToSubtext(text,TT['train'])
    Test  = portionsToSubtext(text,TT['test' ])
    
    X_train =[] 
    y_train =[]

    for i in range(len (Train)):
        y_train.append(Train[i][1:4])
        X_train.append(createFeature(i,Train[i],TT))
        
    X_test =[] 
    y_test =[]

    for i in range(len(Test)):
        y_test.append(Test[i][1:4])
        X_test.append(createFeature(i,Test[i],TT))

    clf=RandomForestClassifier(n_estimators=1000)
    clf.fit(X_train,y_train)
    
    y_pred=clf.predict(X_test)
    TT['score']=metrics.accuracy_score(y_test, y_pred)

In [ ]:
i = 1
for each in Train_Test:
    print("Random Forest Classification with Cross Validation Portion",i, 'score',round(each['score']*100,2),'%')
    i+=1

In [170]:
def export(predictionFunction, tagName = None):
    
    BEST_TRAINED_PARTITION = max( Train_Test, key=lambda i:i['score'])
    #print(round(BEST_TRAINED_PARTITION['score']*100,2),'%')
    
    export = open("EPSI M2-ISD Valery & Charles predictions"+tagName+".txt", "wt")
    file   = open("test.txt","r+")
    
    for line in file:
        export.write('('+ (predictionFunction(line[5:-1],BEST_TRAINED_PARTITION,False))+')'+line[5:-1])
        
    export.close()
    file.close()

In [171]:
#To Create Inline Prediction File
export(box)

42.12 %
